In [78]:
import csv
import pandas as pd
import nltk
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing

from pythainlp.tokenize import Tokenizer
from pythainlp import word_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.tokenize import dict_trie

from nltk.corpus import words
from stop_words import get_stop_words

# Load data

In [79]:
file_name = 'category1_data-clean-utf8'
df = pd.read_excel('./Data/{}.xlsx'.format(file_name), index_col=0).fillna('-').reset_index()

In [80]:
df.head()

Column1  index category_0_0_label  \
0        0      0                cat   
1        1      1                sub   
2        2      2               name   
3        3      3                  o   
4        4      4               attr   

                                        category_0_0 category_0_1_label  \
0                          โทรศัพท์มือถือและแท็บเล็ต                cat   
1                                        smartphones                sub   
2         samsung galaxy m30s 4 64gb โทรศัพท์มือถือ                name   
3  แบตอึดอยู่นาน สนุกได้ทั้งวัน กับความจุ 6000 ma...                  o   
4  ระบบกล้องหลัง 3 ตัว ที่มีกล้องชัดสูงสุดถึง 48 ...                  o   

                                        category_0_1 category_0_2_label  \
0                          โทรศัพท์มือถือและแท็บเล็ต                cat   
1                                        smartphones                sub   
2  โทรศัพท์มือถือ samsung galaxy a50s ram4 64gb ร...               name   
3                           gsm 850 900 1800 1900mhz                  o   
4                      3g wcdma 850 900 1900 2100mhz               attr   

                                        category_0_2 category_0_3_label  \
0                          โทรศัพท์มือถือและแท็บเล็ต                cat   
1                                        smartphones                sub   
2                samsung galaxy a10s ram2gb rom32gb                name   
3       สมาร์ทโฟน โทรศัพท์มือถือพร้อมระบบปฏิบัติการ                attr   
4  จอแสดงผล tft lcd 24 bit 16 ล้านสี หน้าจอทรงหยด...               attr   

                                       category_0_3  ...  \
0                         โทรศัพท์มือถือและแท็บเล็ต  ...   
1                                       smartphones  ...   
2  samsung galaxy note 10 12 256 gb โทรศัพท์มือถือ   ...   
3                               ขนาดหน้าจอ 6.8 นิ้ว  ...   
4                                         ram 12 gb  ...   

                                      category_0_270  \
0                          โทรศัพท์มือถือและแท็บเล็ต   
1                                        smartphones   
2  new haixu v14 หน้าจอ 5.5 นิ้ว 2sim ram2 rom 16...   
3                   หน้าจอ 5.5 นิ้ว 2sim ram2 rom 16   
4                                        brand haixu   

                                      category_0_271  \
0   ซื้อ 1 แถม 1 ฟรี สำหรับ vivo v9 v11 v11i v7 y...   
1  huawei nova 5t 128gb rom 8gb ram ฿ 5400.00 ฿ 1...   
2          oppo a5 2020 ประกันศูนย์oppo1ปี ฿ 4690.00   
3  apple 7 plus 8 plus โทรศัพท์มือถือ anti ลายนิ้...   
4  เครื่องเกาหลี นำเข้า note9 ฿ 4299.00 ฿ 6999.00...   

                                      category_0_272  \
0                          โทรศัพท์มือถือและแท็บเล็ต   
1                                        smartphones   
2  ฟรีค่าส่งทั้งร้าน haixu star หน้าจอใหญ่ 5.5 นิ...   
3                                 หน้าจอใหญ่ 5.5นิ้ว   
4                                        brand haixu   

                                      category_0_273  \
0                          โทรศัพท์มือถือและแท็บเล็ต   
1                                        smartphones   
2  haixu star edit หน้าจอใหญ่ 5.5 นิ้ว 16gb รองรั...   
3                                 หน้าจอใหญ่ 5.5นิ้ว   
4                                        brand haixu   

                                      category_0_274  \
0                          โทรศัพท์มือถือและแท็บเล็ต   
1                                        smartphones   
2  haixu v11 หน้าจอใหญ่ 6.1 นิ้ว ram2 rom32gb สาม...   
3                                         จอใหญ่ 6.1   
4                                        brand haixu   

                                      category_0_275  \
0                          โทรศัพท์มือถือและแท็บเล็ต   
1                                        smartphones   
2  haixu star 16gb หน้าจอใหญ่ 5.5 battery 3200mah...   
3                    รองรับ 3g 4g ทุกเครือข่าย 2sim    
4                            หน้าจอใหญ่ 5.5 นิ้ว qhd   

                          

In [81]:
# Drop a column that not use
data = df.drop(columns=['Column1','index'])

In [83]:
data.tail()

category_0_0_label category_0_0 category_0_1_label  \
29                  -            -                  o   
30                  -            -                  o   
31                  -            -                  o   
32                  -            -                  -   
33                  -            -                  -   

                             category_0_1 category_0_2_label category_0_2  \
29              กล้องด้านหลัง 21mp ขึ้นไป                  -            -   
30  ความจุแบตเตอรี่ 1000 mah ถึง 5000 mah                  -            -   
31                    ช่องเสียบแบบบาง คู่                  -            -   
32                                      -                  -            -   
33                                      -                  -            -   

   category_0_3_label category_0_3 category_0_4_label  \
29                  -            -                  o   
30                  -            -                  o   
31                  -            -                  o   
32                  -            -                  o   
33                  -            -                  o   

                                         category_0_4  ... category_0_270  \
29                                   ข้อความแชท skype  ...              -   
30                       สนับสนุน gmail google search  ...              -   
31                ไมโครโฟนสำหรับตัดเสียงรบกวนการสนทนา  ...              -   
32  ออร์กาไนเซอร์ ปฏิทิน เครื่องคิดเลข นาฬิกาปลุก ...  ...              -   
33            ริงโทน mp3 ระบบสั่น vibration in phone   ...              -   

   category_0_271 category_0_272 category_0_273 category_0_274 category_0_275  \
29              -              -              -              -              -   
30              -              -              -              -              -   
31              -              -              -              -              -   
32              -              -              -              -              -   
33              -              -              -              -              -   

   category_0_276 category_0_277 category_0_278                 category_0_279  
29              -              -              -  ความจุแบตเตอรี่ 3000 3999 mah  
30              -              -              -                            120  
31              -              -              -                              -  
32              -              -              -                              -  
33              -              -              -                              -  

[5 rows x 314 columns]

In [84]:
i=0
pd_des = []
pd_label = []
while i<34:
    pd_des = np.append(pd_des,data['category_0_{}'.format(i)].values)
    pd_label = np.append(pd_label,data['category_0_{}_label'.format(i)].values)
    i += 1

In [85]:
len(pd_des)

1156

In [86]:
len(pd_label)

1156

In [87]:
x = data.filter(regex='[0-9]$', axis=1) 

In [88]:
x.head()

category_0_0  \
0                          โทรศัพท์มือถือและแท็บเล็ต   
1                                        smartphones   
2         samsung galaxy m30s 4 64gb โทรศัพท์มือถือ    
3  แบตอึดอยู่นาน สนุกได้ทั้งวัน กับความจุ 6000 ma...   
4  ระบบกล้องหลัง 3 ตัว ที่มีกล้องชัดสูงสุดถึง 48 ...   

                                        category_0_1  \
0                          โทรศัพท์มือถือและแท็บเล็ต   
1                                        smartphones   
2  โทรศัพท์มือถือ samsung galaxy a50s ram4 64gb ร...   
3                           gsm 850 900 1800 1900mhz   
4                      3g wcdma 850 900 1900 2100mhz   

                                        category_0_2  \
0                          โทรศัพท์มือถือและแท็บเล็ต   
1                                        smartphones   
2                samsung galaxy a10s ram2gb rom32gb    
3       สมาร์ทโฟน โทรศัพท์มือถือพร้อมระบบปฏิบัติการ    
4  จอแสดงผล tft lcd 24 bit 16 ล้านสี หน้าจอทรงหยด...   

                                       category_0_3  \
0                         โทรศัพท์มือถือและแท็บเล็ต   
1                                       smartphones   
2  samsung galaxy note 10 12 256 gb โทรศัพท์มือถือ    
3                               ขนาดหน้าจอ 6.8 นิ้ว   
4                                         ram 12 gb   

                                        category_0_4  \
0                          โทรศัพท์มือถือและแท็บเล็ต   
1                                        smartphones   
2                            samsung galaxy a7 2017    
3  ระบบเซ็นเซอร์ sensor ระบบตรวจสอบลายนิ้วมือ fin...   
4    คุณสมบัติการกันน้ำ waterproof กันน้ำได้ชั่วคราว   

                                        category_0_5  \
0                          โทรศัพท์มือถือและแท็บเล็ต   
1                                        smartphones   
2  samsung galaxy a9 2018 6 gb ram 128 gb rom 6.3...   
3  จอแสดงผล super amoled 24 bit 16 ล้านสี fullvie...   
4  ระบบตรวจสอบลายนิ้วมือ fingerprint ระบบจดจำใบหน...   

                                        category_0_6  \
0                          โทรศัพท์มือถือและแท็บเล็ต   
1                                        smartphones   
2  samsung a30s ram 4gb rom 64 gb มือ 1 ประกันศูน...   
3                             คุณสมบัติ samsung a30s   
4            จอแสดงผล super amoled 24 bit 16 ล้านสี    

                                        category_0_7  \
0                          โทรศัพท์มือถือและแท็บเล็ต   
1                                        smartphones   
2   มือ1 ศูนย์ไทย samsung galaxy note 10 plus not...   
3  samsung galaxy note10 plus th ประกันศูนย์ไทย 1...   
4                                       brand ซัมซุง   

                                 category_0_8  \
0                   โทรศัพท์มือถือและแท็บเล็ต   
1                                 smartphones   
2  samsung galaxy a70 8 128gb โทรศัพท์มือถือ    
3                  galaxy a70 sm a705fzbgthl    
4                          สเปคเทพของเกมเมอร์   

                                        category_0_9  ...  \
0                          โทรศัพท์มือถือและแท็บเล็ต  ...   
1                                        smartphones  ...   
2  smartphone โทรศัพท์มือถือ samsung galaxy a10s ...  ...   
3                                       brand ซัมซุง  ...   
4                        sku 571250946 th 1048780375  ...   

                                      category_0_270  \
0                          โทรศัพท์มือถือและแท็บเล็ต   
1                                        smartphones   
2  new haixu v14 หน้าจอ 5.5 นิ้ว 2sim ram2 rom 16...   
3                   หน้าจอ 5.5 นิ้ว 2sim ram2 rom 16   
4                                        brand haixu   

                                      category_0_271  \
0   ซื้อ 1 แถม 1 ฟรี สำหรับ vivo v9 v11 v11i v7 y...   
1  huawei nova 5t 128gb rom 8gb ram ฿ 5400.00 ฿ 1...   
2          oppo a5 2020 ประกันศูนย์oppo1ปี ฿ 4690.00   
3  apple 7 plus 8 plus โทรศัพท์มือถือ anti ลายนิ้...   
4  เครื่องเกาหลี นำเข้า note9 ฿ 4299.00 ฿ 6999.00...   

              

In [89]:
x_train = x.iloc[:,0:34]

In [90]:
x_train.head()

category_0_0  \
0                          โทรศัพท์มือถือและแท็บเล็ต   
1                                        smartphones   
2         samsung galaxy m30s 4 64gb โทรศัพท์มือถือ    
3  แบตอึดอยู่นาน สนุกได้ทั้งวัน กับความจุ 6000 ma...   
4  ระบบกล้องหลัง 3 ตัว ที่มีกล้องชัดสูงสุดถึง 48 ...   

                                        category_0_1  \
0                          โทรศัพท์มือถือและแท็บเล็ต   
1                                        smartphones   
2  โทรศัพท์มือถือ samsung galaxy a50s ram4 64gb ร...   
3                           gsm 850 900 1800 1900mhz   
4                      3g wcdma 850 900 1900 2100mhz   

                                        category_0_2  \
0                          โทรศัพท์มือถือและแท็บเล็ต   
1                                        smartphones   
2                samsung galaxy a10s ram2gb rom32gb    
3       สมาร์ทโฟน โทรศัพท์มือถือพร้อมระบบปฏิบัติการ    
4  จอแสดงผล tft lcd 24 bit 16 ล้านสี หน้าจอทรงหยด...   

                                       category_0_3  \
0                         โทรศัพท์มือถือและแท็บเล็ต   
1                                       smartphones   
2  samsung galaxy note 10 12 256 gb โทรศัพท์มือถือ    
3                               ขนาดหน้าจอ 6.8 นิ้ว   
4                                         ram 12 gb   

                                        category_0_4  \
0                          โทรศัพท์มือถือและแท็บเล็ต   
1                                        smartphones   
2                            samsung galaxy a7 2017    
3  ระบบเซ็นเซอร์ sensor ระบบตรวจสอบลายนิ้วมือ fin...   
4    คุณสมบัติการกันน้ำ waterproof กันน้ำได้ชั่วคราว   

                                        category_0_5  \
0                          โทรศัพท์มือถือและแท็บเล็ต   
1                                        smartphones   
2  samsung galaxy a9 2018 6 gb ram 128 gb rom 6.3...   
3  จอแสดงผล super amoled 24 bit 16 ล้านสี fullvie...   
4  ระบบตรวจสอบลายนิ้วมือ fingerprint ระบบจดจำใบหน...   

                                        category_0_6  \
0                          โทรศัพท์มือถือและแท็บเล็ต   
1                                        smartphones   
2  samsung a30s ram 4gb rom 64 gb มือ 1 ประกันศูน...   
3                             คุณสมบัติ samsung a30s   
4            จอแสดงผล super amoled 24 bit 16 ล้านสี    

                                        category_0_7  \
0                          โทรศัพท์มือถือและแท็บเล็ต   
1                                        smartphones   
2   มือ1 ศูนย์ไทย samsung galaxy note 10 plus not...   
3  samsung galaxy note10 plus th ประกันศูนย์ไทย 1...   
4                                       brand ซัมซุง   

                                 category_0_8  \
0                   โทรศัพท์มือถือและแท็บเล็ต   
1                                 smartphones   
2  samsung galaxy a70 8 128gb โทรศัพท์มือถือ    
3                  galaxy a70 sm a705fzbgthl    
4                          สเปคเทพของเกมเมอร์   

                                        category_0_9  ...  \
0                          โทรศัพท์มือถือและแท็บเล็ต  ...   
1                                        smartphones  ...   
2  smartphone โทรศัพท์มือถือ samsung galaxy a10s ...  ...   
3                                       brand ซัมซุง  ...   
4                        sku 571250946 th 1048780375  ...   

                                       category_0_24  \
0                          โทรศัพท์มือถือและแท็บเล็ต   
1                                        smartphones   
2  samsung galaxy a9 2018 6 128gb เครื่องศูนย์ เค...   
3  จอแสดงผล super amoled 24 bit 16 ล้านสี fullvie...   
4  ระบบตรวจสอบลายนิ้วมือ fingerprint ระบบจดจำใบหน...   

                                       category_0_25  \
0                          โทรศัพท์มือถือและแท็บเล็ต   
1                                        smartphones   
2      samsung a70 ram8 128 เครื่องศูนย์เคลียร์สต๊อค   
3  จอแสดงผล super amoled 24 bit 16 ล้านสี หน้าจอท...   
4                                       brand ซัมซุง   

              

# Tokenize

In [13]:
# download stopwords of TH and EN
nltk.download('words')
th_stop = tuple(thai_stopwords())
en_stop = tuple(get_stop_words('en'))

# path of custom dictionary for word tokenize
PATH_TO_CUSTOM_DICT = './Data/custom_dict.txt'

# split text to words 
def split_word(text):
    
    # Split text to words
    _tokenizer = Tokenizer(custom_dict=PATH_TO_CUSTOM_DICT, engine='newmm')
    words = _tokenizer.word_tokenize(text)

    # Remove stop words in TH and EN
    words = [i for i in words if not i in th_stop and not i in en_stop]
    
    # Delete whitespace
    words = [i for i in words if not ' ' in i]

    return words

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\super\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [14]:
token = {}
for key,value in x_train.items():
    tmp = [split_word(txt) for txt in value if not txt == '-']
    token[key] = tmp

In [15]:
token

{'category_0_0': [['โทรศัพท์มือถือและแท็บเล็ต'],
  ['smartphones'],
  ['samsung', 'galaxy', 'm', '30', 's', '4', '64', 'gb', 'โทรศัพท์มือถือ'],
  ['แบต',
   'อึดอยู่นาน',
   'สนุกได้ทั้งวัน',
   'กับความจุ',
   '6000',
   'mah',
   'และสามารถชาร์จ',
   'แบต',
   'fast',
   'charging'],
  ['ระบบ',
   'กล้องหลัง',
   '3',
   'ตัว',
   'ที่มีกล้องชัดสูงสุดถึง',
   '48',
   'mp',
   'มาพร้อมกับเลนส์มุมกว้างพิเศษ',
   'ultra',
   'wide'],
  ['exynos',
   '9611',
   'สเป็กแรงขึ้นเล่นเกมส์ได้ไม่มีสะดุด',
   'ram',
   '4',
   'gb',
   'rom',
   '64',
   'gb'],
  ['หน้าจอแสดงผล', 'super', 'amoled', 'infinity', 'u', 'ขนาด', '6.4', 'นิ้ว'],
  ['brand', 'ซัมซุง'],
  ['sku', '480942616', 'th', '870640886'],
  ['ประเภทของการรับประกัน', 'มีการรับประกัน'],
  ['model', 'galaxy', 'm', '30', 's', '4', '64', 'gb'],
  ['ระยะเวลาการรับประกัน', '1', 'ปี'],
  ['793']],
 'category_0_1': [['โทรศัพท์มือถือและแท็บเล็ต'],
  ['smartphones'],
  ['โทรศัพท์มือถือ',
   'samsung',
   'galaxy',
   '50',
   's',
   'ram',

# Create BOW and train-test split

In [91]:
tokens_list = []
for i,v in token.items():
    tmp = [','.join(tkn) for tkn in v]
    tokens_list.append(tmp)

In [92]:
tokens_list

[['โทรศัพท์มือถือและแท็บเล็ต',
  'smartphones',
  'samsung,galaxy,m,30,s,4,64,gb,โทรศัพท์มือถือ',
  'แบต,อึดอยู่นาน,สนุกได้ทั้งวัน,กับความจุ,6000,mah,และสามารถชาร์จ,แบต,fast,charging',
  'ระบบ,กล้องหลัง,3,ตัว,ที่มีกล้องชัดสูงสุดถึง,48,mp,มาพร้อมกับเลนส์มุมกว้างพิเศษ,ultra,wide',
  'exynos,9611,สเป็กแรงขึ้นเล่นเกมส์ได้ไม่มีสะดุด,ram,4,gb,rom,64,gb',
  'หน้าจอแสดงผล,super,amoled,infinity,u,ขนาด,6.4,นิ้ว',
  'brand,ซัมซุง',
  'sku,480942616,th,870640886',
  'ประเภทของการรับประกัน,มีการรับประกัน',
  'model,galaxy,m,30,s,4,64,gb',
  'ระยะเวลาการรับประกัน,1,ปี',
  '793'],
 ['โทรศัพท์มือถือและแท็บเล็ต',
  'smartphones',
  'โทรศัพท์มือถือ,samsung,galaxy,50,s,ram,4,64,gb,รับประกันศูนย์,jaymart',
  'gsm,850,900,1800,1900,mhz',
  '3,g,wcdma,850,900,1900,2100,mhz',
  '4,g,fdd,lte,b,1,2100,b,3,1800,b,5,850,b,7,2600,b,8,900,b,20,800,b,28,700',
  '4,g,tdd,lte,b,38,2600,b,40,2300,b,41,2500',
  'รองรับ,2,ซิม,nano,sim',
  'หน่วยประมวลผล,exynos,9610,quad,2.3,ghz,quad,1.7,ghz',
  'ระบบปฏิบัติการ,android',

In [93]:
for i,v in token.items():
    tokens_list_j = [','.join(tkn) for tkn in tokens_list]

cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit(tokens_list_j)

In [97]:
# bag of words
cvec.vocabulary_

{'โทรศัพท์มือถือและแท็บเล็ต': 938,
 'smartphones': 542,
 'samsung': 524,
 'galaxy': 374,
 'm': 433,
 '30': 117,
 's': 521,
 '4': 141,
 '64': 212,
 'gb': 376,
 'โทรศัพท์มือถือ': 936,
 'แบต': 921,
 'อึดอยู่นาน': 866,
 'สนุกได้ทั้งวัน': 811,
 'กับความจุ': 629,
 '6000': 206,
 'mah': 434,
 'และสามารถชาร์จ': 927,
 'fast': 353,
 'charging': 315,
 'ระบบ': 770,
 'กล้องหลัง': 627,
 '3': 114,
 'ตัว': 692,
 'ที่มีกล้องชัดสูงสุดถึง': 703,
 '48': 162,
 'mp': 447,
 'มาพร้อมกับเลนส์มุมกว้างพิเศษ': 753,
 'ultra': 581,
 'wide': 603,
 'exynos': 349,
 '9611': 264,
 'สเป็กแรงขึ้นเล่นเกมส์ได้ไม่มีสะดุด': 830,
 'ram': 510,
 'rom': 520,
 'หน้าจอแสดงผล': 853,
 'super': 556,
 'amoled': 276,
 'infinity': 408,
 'u': 577,
 'ขนาด': 639,
 '6.4': 202,
 'นิ้ว': 708,
 'brand': 304,
 'ซัมซุง': 679,
 'sku': 534,
 '480942616': 163,
 'th': 567,
 '870640886': 253,
 'ประเภทของการรับประกัน': 722,
 'มีการรับประกัน': 755,
 'model': 445,
 'ระยะเวลาการรับประกัน': 790,
 '1': 1,
 'ปี': 732,
 '793': 239,
 '50': 171,
 'รับประกันศูนย์

In [98]:
# จะแบ่ง train test split ข้อมูล แล้วเอาไป fit กับ bag of words ที่สร้างไว้ เพื่อแปลงเป็นตัวเลขก่อนเอาไปเข้าโมเดล
# https://realpython.com/python-keras-text-classification/

des_train, des_test, y_train, y_test = train_test_split(pd_des, pd_label, test_size=0.1, random_state=47)

In [99]:
X_train = c_feat.transform(des_train)
X_test  = c_feat.transform(des_test)

In [100]:
le = preprocessing.LabelEncoder()
le.fit(y_train)
Y_train = le.transform(y_train)
Y_test = le.transform(y_test)

In [102]:
list(le.classes_)

['-', 'attr', 'cat', 'name', 'o', 'sku', 'sub']

# Logistic regression model

In [103]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, Y_train)
score = classifier.score(X_test, Y_test)

print("Accuracy:", score)

Accuracy: 0.5517241379310345


C:\Users\super\Anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\super\Anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [15]:
# Save to csv file (label)
#pd.DataFrame.from_dict(label,orient='index').transpose().to_csv('./Data/536-585-label.csv')

In [29]:
# Save to csv file (word)
#pd.DataFrame.from_dict(groupProduct,orient='index').transpose().to_csv('./Data/536-585-processed.csv')